# 03. Inference - Terms Extract
This tutorial shows you how to use LLMs to extract terms from text with Semantic Kernel.

## Prerequisites

Refer to [01-get-started.ipynb](./01-get-started.ipynb) to install Ollama.

## Install Semantic Kernel

Run the following cell to install the packages.

In [3]:
#r "nuget: Microsoft.SemanticKernel, 1.11.1"

Installed Packages Microsoft.SemanticKernel, 1.11.1

## Instantiate the kernel

In [22]:
#pragma warning disable SKEXP0010
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

// We use Mistral for this tutorial 
var modelId = "mistral";
// local Ollama endpoint
var endpoint = new Uri("http://localhost:11434");

var kernelBuilder = Kernel.CreateBuilder();
var kernel = kernelBuilder
    .AddOpenAIChatCompletion(
        modelId,
        endpoint,
        apiKey:null) 
    .Build();

#pragma warning restore SKEXP0010

##  Example code

The following example creates a prompt template and then extract table names that will be used for data analytics.

In [23]:
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;

var chatService = kernel.GetRequiredService<IChatCompletionService>();

var executionSettings = new OpenAIPromptExecutionSettings
{
    MaxTokens = 2000,
    Temperature = 0,
};

var skPrompt = """

You are a data analytics AI assistant from Kontext.

In a Snowflake database you are working with, the following tables \
were created with the following schema:

```
CREATE TABLE "Customers" (
    "CustomerId" INTEGER NOT NULL, 
    "FirstName" NVARCHAR(40) NOT NULL, 
    "LastName" NVARCHAR(20) NOT NULL, 
    PRIMARY KEY ("CustomerId"),
)

CREATE TABLE "Accounts" (
    "AccountId" INTEGER NOT NULL, 
    "Balance" DECIMAL NOT NULL,
    PRIMARY KEY ("AccountId"),
)

CREATE TABLE "CustomerAccounts" (
    "CustomerId" INTEGER NOT NULL, 
    "AccountId" INTEGER NOT NULL, 
    PRIMARY KEY ("CustomerId", "AccountId"),
    FOREIGN KEY("CustomerId") REFERENCES "Customers" ("CustomerId")
    FOREIGN KEY("AccountId") REFERENCES "Accounts" ("AccountId")
)

```

You need to decide the table(s) that will be required to answer the \
following question, which is delimited with triple backticks. 

Provide your answer with the required table name(s) as an JSON array ONLY.

Question: ```{{$question}}```
""";


In [24]:
// Now let's directly invoke the prompt template with argument to get the response.

var getTableNamesAsync = async (string question) =>
{
    return await kernel.InvokePromptAsync<string>(skPrompt, new() { ["question"] = question });
};

display(await getTableNamesAsync("What is the first name of the customer with ID 123?"));
display(await getTableNamesAsync("How many accounts does the customer with ID 123 have?"));
display(await getTableNamesAsync("What is the total balance of customer with ID 123?"));

 ["Customers"]

 ["CustomerAccounts"]

 ["Customers", "CustomerAccounts", "Accounts"]